In [1]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import linear_sum_assignment
import numpy as np
from sklearn.covariance import MinCovDet
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import predictive_clustering,utils,predictive_plots,predictive_validation

## Detecting Outliers Clusters

In [2]:
baseline = pd.read_excel("RETOS_BEBRASK_Baseline.xlsx")
scales = pd.read_excel("filled_scales_BEBRASK_RETOS.xlsx")
scaled_df = pd.read_excel("baseline_scaled.xlsx")
scaled_transformed_df = pd.read_excel("count_scaled_transformed.xlsx")


### Baseline

In [16]:
baseline

,Subject,Mean_Rating0,Dif_Match,Cor_Pred_Like
0,PREDWELL_RETOS-1-1,2.888889,1.614379,0.113495
1,PREDWELL_RETOS-10-1,2.533333,1.117647,0.214781
2,PREDWELL_RETOS-1001-1,2.644444,1.494553,0.336312
3,PREDWELL_RETOS-1002-1,2.711111,1.318083,0.450822
4,PREDWELL_RETOS-1003-1,2.200000,1.424837,0.073526
...,...,...,...,...
144,PREDWELL_RETOS-5-1,2.377778,1.300654,0.225374
145,PREDWELL_RETOS-6-1,2.266667,1.152505,0.499722
146,PREDWELL_RETOS-7-1,2.444444,1.352941,0.426911
147,PREDWELL_RETOS-8-1,2.555556,1.117647,0.186173


In [17]:
import plotly.graph_objects as go
import pandas as pd
from sklearn.covariance import MinCovDet

# Assuming 'baseline' is your DataFrame and it has been imported already

# Drop the 'Subject' column and store the result for reuse
data_without_subject = baseline.drop("Subject", axis=1)

# Fit MinCovDet to the data without the 'Subject' column
robust_cov = MinCovDet().fit(data_without_subject)

# Calculate the Mahalanobis distance for each point
mahalanobis_distance = robust_cov.mahalanobis(data_without_subject)
# Define a threshold for outliers as the 95th percentile of these distances
threshold = np.percentile(mahalanobis_distance, 95)
outliers_baseline = mahalanobis_distance > threshold

# Create a Plotly figure
fig = go.Figure()

# Add normal data points
fig.add_trace(go.Scatter3d(
    x=data_without_subject['Mean_Rating0'],
    y=data_without_subject['Dif_Match'],
    z=data_without_subject['Cor_Pred_Like'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue',  # normal points in blue
    ),
    name='Data Points'
))

# Add detected outliers
fig.add_trace(go.Scatter3d(
    x=data_without_subject.loc[outliers_baseline, 'Mean_Rating0'],
    y=data_without_subject.loc[outliers_baseline, 'Dif_Match'],
    z=data_without_subject.loc[outliers_baseline, 'Cor_Pred_Like'],
    mode='markers',
    marker=dict(
        size=8,
        color='red',  # outliers in red
        symbol='cross'
    ),
    name='Detected Outliers'
))

# Update plot layout
fig.update_layout(
    title='Outlier Detection with Minimum Covariance Determinant',
    scene=dict(
        xaxis_title='Mean_Rating0',
        yaxis_title='Dif_Match',
        zaxis_title='Cor_Pred_Like'
    ),
    legend_title_text='Legend'
)

fig.show()



### Count


In [18]:
data_without_subject_count = scaled_transformed_df.drop("Subject", axis=1)

# Fit MinCovDet to the data without the 'Subject' column
robust_cov = MinCovDet().fit(data_without_subject_count)

# Calculate the Mahalanobis distance for each point
mahalanobis_distance = robust_cov.mahalanobis(data_without_subject_count)
# Define a threshold for outliers as the 95th percentile of these distances
threshold = np.percentile(mahalanobis_distance, 95)
outliers_count = mahalanobis_distance > threshold

fig = go.Figure()

# Add normal data points
fig.add_trace(go.Scatter3d(
    x=data_without_subject['Mean_Rating0'],
    y=data_without_subject['Dif_Match'],
    z=data_without_subject['Cor_Pred_Like'],
    mode='markers',
    marker=dict(
        size=5,
        color='blue',  # normal points in blue
    ),
    name='Data Points'
))

# Add detected outliers
fig.add_trace(go.Scatter3d(
    x=data_without_subject.loc[outliers_count, 'Mean_Rating0'],
    y=data_without_subject.loc[outliers_count, 'Dif_Match'],
    z=data_without_subject.loc[outliers_count, 'Cor_Pred_Like'],
    mode='markers',
    marker=dict(
        size=8,
        color='red',  # outliers in red
        symbol='cross'
    ),
    name='Detected Outliers'
))

# Update plot layout
fig.update_layout(
    title='Outlier Detection with Minimum Covariance Determinant',
    scene=dict(
        xaxis_title='Mean_Rating0',
        yaxis_title='Dif_Match',
        zaxis_title='Cor_Pred_Like'
    ),
    legend_title_text='Legend'
)

fig.show()



In [19]:
baseline.loc[outliers_baseline]

,Subject,Mean_Rating0,Dif_Match,Cor_Pred_Like
5,PREDWELL_RETOS-1004-1,2.044444,0.420479,0.430767
43,PREDWELL_RETOS-1104-1,2.400000,0.607843,0.630761
57,PREDWELL_RETOS-1117-1,3.066667,0.973856,0.219610
81,PREDWELL_RETOS-123-1,2.244444,0.657952,0.052960
90,PREDWELL_RETOS-20-1,2.244444,-0.013072,0.359682
96,PREDWELL_RETOS-206-1,2.066667,0.457516,0.726111
123,PREDWELL_RETOS-4-1,1.800000,0.949891,0.307895
133,PREDWELL_RETOS-410-1,2.400000,0.666667,0.139461


In [20]:
baseline.loc[outliers_count]

,Subject,Mean_Rating0,Dif_Match,Cor_Pred_Like
0,PREDWELL_RETOS-1-1,2.888889,1.614379,0.113495
2,PREDWELL_RETOS-1001-1,2.644444,1.494553,0.336312
43,PREDWELL_RETOS-1104-1,2.400000,0.607843,0.630761
57,PREDWELL_RETOS-1117-1,3.066667,0.973856,0.219610
62,PREDWELL_RETOS-1121-1,2.200000,1.233115,0.538314
90,PREDWELL_RETOS-20-1,2.244444,-0.013072,0.359682
100,PREDWELL_RETOS-21-1,2.577778,1.287582,0.818175
137,PREDWELL_RETOS-414-1,2.355556,1.093682,0.071288
